In [1]:
method_name = c('MHU', 'GWG', 'NCG', 'AVG', 'V-DHAMS', 'O-DHAMS')
display_names = c('Metropolis', 'GWG', 'NCG', 'AVG',  'V-DHAMS','O-DHAMS')
source('algos.R')
prob_tables = readRDS('probtable.RData')

Loading required package: pracma

Loading required package: Rcpp



Loading required package: RcppArmadillo



In [2]:
#--- Specify the directory to store the plots
wd = getwd()

In [ ]:
#--- nsample: number of draws per chain
#    nplot: number of draws for trace plots
#    ngaps: interval (gap) between draws to calculate metrics
#    nrepeat: number of repeated chains
#    nsize: (2*nsize+1) is the lattice size

nsample = 15000
nplot = 450
ngaps = 300
nrepeat = 100
nsize = 10

In [4]:
#--- ESS functions 
ESS_2 = function(xss_multi){
  ndim = dim(xss_multi)[2]
  n = dim(xss_multi)[3]
  m = dim(xss_multi)[1]
  ess_2 = vector(length = ndim)
  for(i in 1:ndim){
    xss = xss_multi[,i,]
    row_mean = rowMeans(xss)
    W = 1/(m*(n-1))*sum(sweep(xss, 1, row_mean)^2)
    B = n/(m-1)*sum((row_mean-mean(xss))^2)
    ess_2[i] = n*W/B
  }
  return(ess_2)
}

ess_single = function(xs_energy){
    m = nrow(xs_energy)
    n = ncol(xs_energy)
    row_mean = rowMeans(xs_energy)
    W = 1/(m*(n-1))*sum(sweep(xs_energy, 1, row_mean)^2)
    B = n/(m-1)*sum((row_mean-mean(xs_energy))^2)
    ess = n*W/B
    return(ess)
}
ns = ngaps*(1:(nsample/ngaps))
sample_interval = ns



In [5]:
ave_accs = vector(length=length(method_name))
ave_tv1 = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_tv2 = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_tv4 = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_tv1_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_tv2_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_tv4_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))

ave_ex_bias = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_ex2_bias = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_ex12_bias = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_ex_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_ex2_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))
ave_ex12_var = matrix(vector(length = length(method_name)*length(sample_interval)), length(method_name), length(sample_interval))

ess2 = matrix(vector(length = length(method_name)*3), length(method_name), 3)
xl = c(-nsize, nsize)
yl = c(-nsize, nsize)

optimal_params = list()
ess2 = matrix(vector(length = length(method_name)*3), length(method_name), 3)
xl = c(-nsize, nsize)
yl = c(-nsize, nsize)

for(i in 1:length(method_name)){
  re = readRDS(paste(getwd(), '/disgau', method_name[i], '.RData', sep=''))
  ave_accs[i] = re[[2]]
  ave_tv1[i,] = re[[3]]
  ave_tv2[i,] = re[[4]]
  ave_ex_bias[i,] = re[[5]]
  ave_ex_var[i,] = re[[6]]
  ave_ex2_bias[i,] = re[[7]]
  ave_ex2_var[i,] = re[[8]]
  ave_ex12_bias[i,] = re[[9]]
  ave_ex12_var[i,] = re[[10]]
  ave_tv1_var[i,] = re[[11]]
  ave_tv2_var[i,] = re[[12]]
  ave_tv4[i,] = re[[13]]
  ave_tv4_var[i,] = re[[14]]
  ess2_re = ESS_2(re[[1]][,1,,])
  ess2[i,] = c(min(ess2_re), median(ess2_re), max(ess2_re))
  
  optimal_params[[i]] = re[[18]]

  #--- Trace plots of each sampler
  png(paste(wd, '/trace', display_names[i], '.png', sep = ''), 
    width = 1600, height = 1200, res = 200)
    par(mfrow = c(3,3), mar = c(4, 4, 2, 2))

    for (j in 1:9) {
      x_vals <- re[[1]][j,1,1,1:nplot] + 0.1 * rnorm(nplot)
      y_vals <- re[[1]][j,1,2,1:nplot] + 0.1 * rnorm(nplot)
    
      plot(x_vals, y_vals, type = 'l',
       xlim = xl, ylim = yl,
       cex.lab = 1.5, xlab = '', ylab='')
    }

  dev.off()
}



In [6]:
# --- Plots:
#     plot_tv1        : Averaged mean of TV distance for one-dimensional marginal distributions
#     plot_tv2        : Averaged mean of TV distance for two-dimensional marginal distributions
#     plot_tv4        : Averaged mean of TV distance for four-dimensional marginal distributions
#     plot_tv1_var    : Averaged standard deviation of TV distance for one-dimensional marginals
#     plot_tv2_var    : Averaged standard deviation of TV distance for two-dimensional marginals
#     plot_tv4_var    : Averaged standard deviation of TV distance for four-dimensional marginals
#     plot_ex1_bias   : Averaged squared bias in estimating E[x_i]
#     plot_ex1_var    : Averaged variance in estimating E[x_i]
#     plot_ex2_bias   : Averaged squared bias in estimating E[x_i²]
#     plot_ex2_var    : Averaged variance in estimating E[x_i²]
#     plot_ex12_bias  : Averaged squared bias in estimating E[x_i x_j]
#     plot_ex12_var   : Averaged variance in estimating E[x_i x_j]

colors_idx = 1:length(display_names)
sample_interval = ngaps*(1:(nsample/ngaps)) 
png(paste(wd, "/plot_tv1.png", sep = ''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv1[1,], ylim = c(0.0, 0.3), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Mean of TV distance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_tv1[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_tv2.png",sep= ''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv2[1,], ylim = c(0.05, 0.45), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Mean of TV distance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_tv2[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_tv1_var.png", sep= ''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv1_var[1,], ylim = c(0, 0.1), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'SD of TV distance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_tv1_var[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_tv2_var.png", sep =''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv2_var[1,], ylim = c(0.0, 0.1), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'SD of TV distance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_tv2_var[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_ex_bias.png", sep =''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex_bias[1,], ylim = c(0.0, 0.2), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average squared bias')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex_bias[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_ex_var.png", sep=''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex_var[1,], ylim = c(0.0, 4), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average variance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex_var[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_ex2_bias.png", sep =''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex2_bias[1,], ylim = c(0.0, 0.8), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average squared bias')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex2_bias[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_ex2_var.png", sep =''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex2_var[1,], ylim = c(0.0, 10), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average variance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex2_var[i,], col = colors_idx[i], lwd = 2.5)
}
legend('bottomleft', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd, "/plot_ex12_bias.png", sep=''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex12_bias[1,], ylim = c(0, 0.8), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average squared bias')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex12_bias[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()

png(paste(wd,"/plot_ex12_var.png", sep=''), width = 1600, height = 1200, res = 200)
plot(ns, ave_ex12_var[1,], ylim = c(0.0, 10), col = colors_idx[1], type = 'l',
     lwd = 2.5, cex.lab = 1.4, cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Average variance')
for(i in 2:length(colors_idx)) {
  lines(ns, ave_ex12_var[i,], col = colors_idx[i], lwd = 2.5)
}
legend('bottomleft', legend = display_names, col = colors_idx,
       lwd = 2.5, cex = 1.0, x.intersp = 1, y.intersp = 0.85)
dev.off()


png(paste(wd, "/plot_tv4.png", sep=''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv4[1,],
     ylim = c(0.35, 0.9),
     col = colors_idx[1],
     type = 'l',
     lwd = 2.5,
     cex.lab = 1.4,
     cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'Mean of TV distance')
for (i in 2:length(colors_idx)) {
  lines(ns, ave_tv4[i,], col = colors_idx[i], lwd = 2.5)
}
legend('topright',
       legend = display_names,
       col = colors_idx,
       lwd = 2.5,
       cex = 1.0,
       x.intersp = 1,
       y.intersp = 0.85)

dev.off()


png(paste(wd, "/plot_tv4_var.png", sep=''), width = 1600, height = 1200, res = 200)
plot(ns, ave_tv4[1,],
     ylim = c(0.06, 0.12),
     col = colors_idx[1],
     type = 'l',
     lwd = 2.5,
     cex.lab = 1.4,
     cex.axis = 1.2,
     xlab = 'Number of draws',
     ylab = 'SD of TV distance')
for (i in 2:length(colors_idx)) {
  lines(ns, sqrt(ave_tv4_var[i,]), col = colors_idx[i], lwd = 2.5)
}
legend('topright',
       legend = display_names,
       col = colors_idx,
       lwd = 2.5,
       cex = 1.0,
       x.intersp = 1,
       y.intersp = 0.85)

dev.off()

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

...

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

In [7]:
#--- Table of minimum, median and maximum of ESS across coordinate for each sample
rownames(ess2) = display_names
ess2

Metropolis,4.672827,4.723429,4.806474
GWG,5.695278,6.012534,6.264694
NCG,58.503608,58.965299,59.547089
AVG,43.022052,43.671993,43.942492
V-DHAMS,73.874787,75.087502,76.135479
O-DHAMS,82.249031,82.726947,83.776725


In [8]:
#--- Optimal parameters for each sample
names(optimal_params) = display_names
optimal_params

$Metropolis
$Metropolis[[1]]
$Metropolis[[1]]$r
[1] 2



$GWG
$GWG[[1]]
$GWG[[1]]$r
[1] 2



$NCG
$NCG[[1]]
$NCG[[1]]$a
[1] 3.5



$AVG
$AVG[[1]]
$AVG[[1]]$a
[1] 1.88



$`V-DHAMS`
$`V-DHAMS`[[1]]
$`V-DHAMS`[[1]]$a
[1] 0.9

$`V-DHAMS`[[1]]$b
[1] 0.9

$`V-DHAMS`[[1]]$gamma
[1] 1

$`V-DHAMS`[[1]]$c
[1] -0.5



$`O-DHAMS`
$`O-DHAMS`[[1]]
$`O-DHAMS`[[1]]$a
[1] 0.75

$`O-DHAMS`[[1]]$b
[1] 0.9

$`O-DHAMS`[[1]]$gamma
[1] 0.1

$`O-DHAMS`[[1]]$c
[1] -0.7

In [9]:
#--- Acceptance rates for each sample
names(ave_accs) = display_names
ave_accs

Metropolis        GWG        NCG        AVG    V-DHAMS    O-DHAMS 
 0.7322787  0.7120440  0.6070807  0.5818713  0.8632793  0.7942600

In [10]:
#--- Display the ESS of f(s) for each sampler
for(i in 1:length(method_name)){
  re = readRDS(paste(getwd(), '/disgau', method_name[i], '.RData', sep=''))
  energies = apply(re[[1]][,1,,], c(1,3), energy)
  print(paste(method_name[i], ess_single(energies)))
}



[1] "MHU 180.496750670107"
[1] "GWG 10.1215226546408"
[1] "NCG 3388.4756491011"
[1] "AVG 2254.73582630241"
[1] "V-DHAMS 3841.0879101722"
[1] "O-DHAMS 3167.07102160683"


In [11]:
#--- ACF plots for each sampler
for(i in 1:length(method_name)){
  re = readRDS(paste(wd, '/disgau', method_name[i], '.RData', sep=''))
  energies = apply(re[[1]][,1,,], c(1,3), energy)
  png(paste('/home/yz909/dis_Hams_present/output/dis_Gaussian/acf', display_names[i], '.png', sep = ''))
  acf(energies[1, ], lag.max = 30, main= '')
  dev.off()
}

In [12]:
#--- Probability plot of the true one-dimensional marginal distribution 

png(paste(wd, '/freq_actual.png', sep=''), width = 1600, height = 1200, res = 200)
plot(-nsize:nsize, as.numeric(prob_tables[[1]]), xlab ='', ylab = '', type = "l", ylim = c(0.0, 0.15))
dev.off()

png 
  2

In [13]:
#--- Frequency plots of emprical one-dimensional distribution from each sampler

for(i in 1:length(method_name)){
  ndraw = 6500
  re = readRDS(paste(wd, '/disgau', method_name[i], '.RData', sep=''))
  sample = re[[1]][1,1,1,1:ndraw]
  freq = table(sample)/ndraw
  png(paste('/home/yz909/dis_Hams_present/output/dis_Gaussian/freq', display_names[i], '.png', sep = ''), width = 1600, height = 1200, res = 200)

  plot(as.numeric(names(freq)), as.numeric(freq), type = "l",
      ylim = c(0, 0.15), xlab='', ylab = '')

  # Add remaining plots
  for (j in 15:28) {
      sample = re[[1]][j,1,1,1:ndraw]
    freq <- table(sample)/ndraw
    lines(as.numeric(names(freq)), as.numeric(freq))
  }
  dev.off()
}